In [7]:
import pandas as pd
labelword_list = []
for trait in ['A','C','E', 'N', 'O']:
    f_word = open(trait+'_words.txt', 'r')
    labelword_list += f_word.readline().split(',')
    labelword_list += f_word.readline().split(',')
    f_word.close()
print(len(labelword_list))
labelword_set = set(labelword_list)
print(len(labelword_set))

8229
1433


In [11]:
import string
punctuation = string.punctuation
import nltk
import re

data_path = [
    '../data/FriendsPersona/Friends_A_whole.tsv',
    '../data/myPersonality/MyPersonality_A_whole.tsv',
    '../data/pan2015/Pan_A_whole.tsv',
    '../data/Essay/Essay_A_whole.tsv',
    '../data/Kaggle_mbti/mbti_I_whole.tsv',
    
]

for data in data_path:
    df = pd.read_csv(data,  sep='\t')
    df['utterance'] = df['utterance'].apply(lambda x: x.lower())

    def sent_tokenize(df,sent_method):
        sents = []
        for index,row in df.iterrows():
            sent = []
            tempSent = row['utterance'].split('|||')
            if sent_method == 'nltk':
                for i in tempSent:
                    sent = sent + nltk.sent_tokenize(i)
            elif sent_method == 'comma':
                for i in tempSent:
                    sent = sent + re.split(r"[,,.,?,!,;]",i)
            sents = sents + sent
        return sents

    sents = sent_tokenize(df,'comma')
    df_sents = pd.DataFrame(sents)

    # tokenize function
    def text_tokenize(text,tokenize_method):
        # args: 
        #   text: the text need to be tokenize
        #   tokenize_method: method use for tokenize
        ltext = text.lower() # to lowercase
        translab = str.maketrans(punctuation,' '*len(punctuation))
        text_without_punctuation = ltext.translate(translab) # remove punctuation
        if tokenize_method == 'nltk':
            word_split = nltk.word_tokenize(text_without_punctuation) # word tokenize
        return word_split

    # generate tokens
    import nltk
    tokens = []
    for sent in sents:
        token = text_tokenize(sent,'nltk')
        tokens.append(token)


    df_processed = pd.DataFrame(columns=['sent_id','sentence','label_word'])
    for index,item in enumerate(tokens):
        for i in item:
            if i in label_set:
                df_processed = df_processed.append({'sent_id':'sent_'+str(index), 'sentence':sents[index], 'label_word':i},ignore_index=True)


    def get_span(pos, row):
        if pos == 1:
            return row['sentence'].split(row['label_word'])[0]
        elif pos == 2:
            try:
                return row['sentence'].split(row['label_word'])[1]
            except:
                return ""

    df_processed['span_1'] = df_processed.apply(lambda x: get_span(1, x), axis=1)
    df_processed['span_2'] = df_processed.apply(lambda x: get_span(2, x), axis=1)

    df_processed.to_csv(data.split('_')[0]+'_map_label_words_comma.tsv', sep='\t')

In [182]:
template_file_name = [
    './data/MyPersonality_map_label_words_comma.tsv',
    './data/Pan_map_label_words_comma.tsv',
    './data/Friends_map_label_words_comma.tsv',
    './data/Essay_map_label_words_comma.tsv',
]

df = pd.DataFrame([])

for name in template_file_name:
    tmp_df = pd.read_csv(name, sep='\t')
    tmp_df['source'] = name.split('_')[0].split('/')[-1] 
    print(tmp_df.shape)
    df = pd.concat([df, tmp_df], 0)
    print(df.shape)

    
df.head()

(605, 5)
(605, 5)
(1302, 5)
(1907, 5)
(81, 5)
(1988, 5)
(7701, 5)
(9689, 5)


/tmp/ipykernel_6049/649521047.py:14: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df = pd.concat([df, tmp_df], 0)


,Unnamed: 0,sent_id,sentence,label_word,source
0,0,sent_94,any kind of alcohol will do,kind,MyPersonality
1,1,sent_158,(Gracias a Sofi por la canci�n tan genial,genial,MyPersonality
2,2,sent_203,<3might have to trade in bastille day celebrat...,quiet,MyPersonality
3,3,sent_238,is becoming sentimental as I work on the notes...,sentimental,MyPersonality
4,4,sent_258,kind of likes getting up early and getting mor...,kind,MyPersonality


In [186]:
df.sort_values(by='label_word').to_csv('total_comma_templates.tsv', sep='\t', index=False)

In [188]:
df['len_span_1'] = df['span_1'].apply(len)
df['len_span_2'] = df['span_2'].apply(len)

print(df['len_span_1'].describe())
print()
print(df['len_span_2'].describe())


count    9689.000000
mean       50.265456
std       149.786865
min         0.000000
25%        12.000000
50%        24.000000
75%        48.000000
max      3504.000000
Name: len_span_1, dtype: float64

count    9689.000000
mean       41.400970
std       135.368033
min         0.000000
25%         4.000000
50%        16.000000
75%        40.000000
max      2961.000000
Name: len_span_2, dtype: float64
